In [1]:
# 6. 모델 매개변수 최적화하기
# 데이터에 매개변수를 최적화하여 모델을 학습, 검증, 테스트
# 학습: epoch 반복 단계에서 모델은 출력을 추측, 추측과 정답 사이의 오류 loss를 계산
# 매개변수에 대한 오류의 도함수(derivative)수집한 뒤, 경사하강법을 사용하여 매개변수들을 최적화(optimize)함

# 6-1. 기본(Pre-requisite) 코드
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()

In [2]:
# 6-2. 하이퍼파라매터(Hyperparameter)
# :모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수
# epoch: 데이터셋을 반복하는 횟수
# batch size: 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수
# learning rate: 각 배치/에폭에서 모델의 매개변수를 조절하는 비율, 값이 작을수록 학습속도 느려짐, 크면 학습 중 예측할 수 없는 동작이 발생할 수 있음
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [3]:
# 6-3. 최적화 단계(Optimization Loop)
# :모델을 학습하고 최적화함.
# 최적화 단계의 각 반복을 epoch이라고 부름. 하나의 에폭은 다음 두 부분으로 구성됨
# 학습 단계(train loop): 학습용 데이터셋을 반복하고 최적의 매개변수로 수렴함
# 검증/테스트 단계(validation/test loop): 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복함

# 학습 단계에서의 개념 몇가지
# 6-4. 손실 함수(loss funcion)
# :획득한 결과와 실제 값 사이의 틀린 정도(degree of dissimilarity) 측정, 이 값이 최소화가 되도록 함.
# 주어진 데이터 샘플을 입력으로 계산한 예측과 정답(label)을 비교하여 손실(loss) 계산
# 손실함수 종류: nn.MSEloss(Mean Square Error, 회귀 문제), nn.NLLLoss(Negative Log Likelihood, 분류 문제),
#             두 개를 합친 nn.CrossEntropyLoss

# 손실함수 초기화
loss_fn = nn.CrossEntropyLoss()   #모델의 출력 logit을 정규화하고 예측 오류 계산

In [4]:
# 6-5. 옵티마이저(Optimizer)
# :확률적 경사하강법(SGD; Stochastic Gradient Descent), 모든 최적화 절차(logic)은 optimizer 객체에 캡슐화 됨.
# SGD 외에 ADAM, RMSProp 등

# 학습하려는 모델의 매개변수와 learning rate 하이퍼파라매터를 등록하여 옵티마이저 초기화
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [12]:
# 6-6. 전체 구현
# :최적화 코드를 반복하여 수행하는 train_loop / 테스트 데이터로 모델의 성능을 측정하는 test_loop

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # prediction과 loss 계산
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # 역전파
        optimizer.zero_grad()  #모델 매개변수의 변화도를 재설정(변화도 더해지기 때문에 중복 계산을 막기 위해 반복할 때마다 0으로 설정)
        loss.backward()        #예측 손실(prediction loss)을 역전파
        optimizer.step()       #역전파 단계에서 수집된 변화도로 매개변수 조정
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5}/{size:>5d}]")
            
            
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [13]:
# 손실함수와 옵티마이저 초기화 후 train_loop와 test_loop에 전달
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
----------------------------
loss: 1.542589  [    0/60000]
loss: 1.565815  [ 6400/60000]
loss: 1.336818  [12800/60000]
loss: 1.301266  [19200/60000]
loss: 1.591815  [25600/60000]
loss: 1.582785  [32000/60000]
loss: 1.483664  [38400/60000]
loss: 1.641002  [44800/60000]
loss: 1.412488  [51200/60000]
loss: 1.350720  [57600/60000]
Test Error: 
 Accuracy: 56.0%, Avg loss: 1.428507 

Epoch 2
----------------------------
loss: 1.500395  [    0/60000]
loss: 1.529548  [ 6400/60000]
loss: 1.287981  [12800/60000]
loss: 1.255119  [19200/60000]
loss: 1.554102  [25600/60000]
loss: 1.544800  [32000/60000]
loss: 1.448195  [38400/60000]
loss: 1.616470  [44800/60000]
loss: 1.378763  [51200/60000]
loss: 1.318980  [57600/60000]
Test Error: 
 Accuracy: 56.7%, Avg loss: 1.392894 

Epoch 3
----------------------------
loss: 1.458547  [    0/60000]
loss: 1.496789  [ 6400/60000]
loss: 1.244469  [12800/60000]
loss: 1.216078  [19200/60000]
loss: 1.523151  [25600/60000]
loss: 1.510967  [32000/60000]
loss: